In [184]:
import numpy as np

def round_matrix(A, r):
    return np.array([[round(element, r) for element in row] for row in A]) 

**Método da Potência**

In [185]:
def power_method(A, x , e = 1e-5, max_i = 100):   

    p = np.argmax(x)
    
    x = x / x[p]
    
    for i in range(max_i):
        y = np.dot(A, x)
        
        v = y[p]
        
        p = np.argmax(y)
        
        if np.linalg.norm(x - y / y[p]) < e: break
        
        x = y / y[p]

    return v, x

A = np.array([[-4, 14, 0], [-5, 13, 0], [-1, 0, 2]])
x = np.array([1, 1, 1])  

eigenvalue, eigenvector = power_method(A, x)

print(eigenvalue)
print(eigenvector)


6.000052316760543
[ 1.          0.71428945 -0.24998691]


**Método da Potência Simétrico**

In [186]:
def simetric_power_method(A, x, e = 1e-5, max_i = 100):    
    
    x = x / np.linalg.norm(x, 2)
    
    for i in range(max_i):
        y = np.dot(A, x)
        
        v = np.dot(x.T,y)
        
        if np.linalg.norm(x - y / np.linalg.norm(y, 2), 2) < e: break
        
        x = y / np.linalg.norm(y, 2)

    return v, x

A = np.array([[4, -1, 1], [-1, 3, -2], [1, -2, 3]])
x = np.array([1, 0, 0]) 

eigenvalue, eigenvector = simetric_power_method(A, x)

print(eigenvalue)
print(eigenvector)


5.999999999650752
[ 0.57735908 -0.57734586  0.57734586]


**Método da Potência Inverso**

In [187]:
def inverse_power_method(A, x, e = 1e-5, max_i = 100):
    q = np.dot(x.T, np.dot(A, x)) / np.dot(x.T, x)
    p = np.argmax(np.abs(x))

    x = x / x[p]
    
    for i in range(max_i):
        y = np.linalg.solve(A - q * np.eye(len(A)), x)
 
        v = y[p]
        
        p = np.argmax(np.abs(y))
        
        if np.linalg.norm(x - y / y[p]) < e: break

        x = y / y[p]

    v = 1/v + q
    return v, x

A = np.array([[-4, 14, 0], [-5, 13, 0], [-1, 0, 2]])
x = np.array([1, 1, 1])  

eigenvalue, eigenvector = inverse_power_method(A, x)

print(eigenvalue)
print(eigenvector)

6.00000171428669
[ 1.          0.71428694 -0.24999543]


**Deflação de Wielandt**

In [188]:
def wielandt(A,  x0, max_i):
    evalues = []
    evectors = []

    A2 = A.copy()

    for i in range(max_i):
        eigenvalue, eigenvector = simetric_power_method(A2, x0)
        evalues.append(eigenvalue)
        evectors.append(eigenvector)

        eigenvector = eigenvector.reshape(-1, 1)
        A2 = A2 - eigenvalue * np.dot(eigenvector, eigenvector.T)

        x0 = np.random.rand(len(x0))
        for v in evectors:
            x0 = x0 - np.dot(x0, v) * v
        x0 = x0 / np.linalg.norm(x0, 2)

    return evalues, evectors

A = np.array([[4, -1, 1], [-1, 3, -2], [1, -2, 3]])
x = np.array([1, -1, 1]) 

eigenvalue, eigenvector = wielandt(A, x, 3)

print(eigenvalue)
print(eigenvector)


[6.000000000000002, 2.9999999998982734, 1.0000000003051783]
[array([ 0.57735027, -0.57735027,  0.57735027]), array([ 0.81649658,  0.40825333, -0.40824325]), array([-1.74693478e-05,  7.07098046e-01,  7.07115516e-01])]


**Método de Householder**

In [189]:
def householder(A):
    n = len(A)
    T = A.copy()

    for k in range(n-2):
        x = T[k+1:, k]
        
        v = x.copy()
        v[0] += np.sign(x[0]) * np.linalg.norm(x)
        v = v / np.linalg.norm(v)

        H = np.eye(n-k-1) - 2.0 * np.outer(v, v)

        T[k+1:, k:] = np.dot(H, T[k+1:, k:])
        T[:, k+1:] = np.dot(T[:, k+1:], H)
    return T

A = np.array([[4.0, 1.0, -2.0, 2.0],
              [1.0, 2.0, 0.0, 1.0],
              [-2.0, 0.0, 3.0, -2.0],
              [2.0, 1.0, -2.0, -1.0]])

T = householder(A)
print(round_matrix(T, 5))


[[ 4.      -3.       0.      -0.     ]
 [-3.       3.33333 -1.66667  0.     ]
 [ 0.      -1.66667 -1.32     0.90667]
 [-0.      -0.       0.90667  1.98667]]


**Método $QR$**

In [190]:
def gram_schmidt(A):
    Q = np.zeros_like(A)
    temp_vector = np.zeros_like(A)

    Q[:, 0] = A[:, 0] / np.linalg.norm(A[:, 0], ord=2)

    for i in range(1, len(A)):
        q = Q[:, :i]
        temp_vector = np.sum(np.sum(q * A[:, i, None], axis=0) * q, axis=1)
        Q[:, i] = A[:, i] - temp_vector
        Q[:, i] /= np.linalg.norm(Q[:, i], ord = 2)
    return Q

def QR(A):
    Q = gram_schmidt(A)
    R = Q.T @ A
    return (Q, R)

def _2x2(A):
    b = -(A[-1, -1] + A[-2, -2])
    c = A[-1, -1] * A[-2, -2] - A[-2, -1] * A[-1, -2]
    d = np.sqrt(b ** 2 - 4 * c)

    return ( -2*c/(b + d), -(b + d)/2) if b > 0 else ( (d - b)/2 , 2*c/(d - b))


def QR_eigenvalues(T, e = 1e-5 , max_i = 100):
    n = len(T)
    v = np.zeros(n)
    A = T.copy()

    for i in range(max_i):
        if np.abs(A[-1, -2]) <= e:
            n -= 1
            v[n] = A[-1, -1]
            A = A[:-1, :-1]

        v1, v2 = _2x2(A)

        if n == 2:
            v[0], v[1] = v1, v2
            break

        p = np.argmin([v1 - A[-1, -1], v2 - A[-1, -1]])

        a = v1 if p == 0 else v2

        I = np.eye(n)
        Q, R = QR(A - a * I)
        A = R @ Q + a * I
    return v

A = np.array([[4.0, 1.0, -2.0, 2.0],
              [1.0, 2.0, 0.0, 1.0],
              [-2.0, 0.0, 3.0, -2.0],
              [2.0, 1.0, -2.0, -1.0]])

T = QR_eigenvalues(householder(A))
print(T)

[ 6.84462111  2.26853141  1.08436446 -2.19751698]


**Decomposição $USV^{T}$**

In [191]:
def USV(T):
    A = T.copy()
    evaluesU, U = np.linalg.eigh(A @ A.T)

    idx = np.argsort(evaluesU)[::-1]
    evaluesU = evaluesU[idx]
    U = U[:, idx]

    i = np.arange(len(A))
    S[i, i] = np.sqrt(evaluesU)

    evaluesV, V = np.linalg.eigh(A.T @ A)

    idx = np.argsort(evaluesV)[::-1]
    evaluesV = evaluesV[idx]
    V = V[:, idx]

    return U, S, V.T

A = np.array([[3, 1.0, 0],
              [1, 3.0, 1],
              [0, 1.0, 3]])

U, S, VT = USV(A)

T = U @ S @ VT

print(round_matrix(T, 5))

[[ 3.  1.  0.]
 [ 1.  3.  1.]
 [-0.  1.  3.]]
